In [2]:
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import requests

In [3]:
from transformers import AutoModel, AutoTokenizer
import torch

In [4]:
def load_and_split_documents(documents_folder):
    """
    Загрузка документов из локальной директории и разбиение их на фрагменты.
    """
    # Загрузка документов с указанием кодировки utf-8
    loader = DirectoryLoader(
        documents_folder, glob="*.txt", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"}
    )
    documents = loader.load()
    
    # Разбиение документов на фрагменты
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    texts = text_splitter.split_documents(documents)
    return texts

In [5]:
def create_embeddings(texts, model_name='DeepPavlov/rubert-base-cased'):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    
    # Токенизация и создание эмбеддингов
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Использование mean pooling для создания эмбеддингов
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

In [13]:
def create_vectorstore(texts):
    """
    Создание эмбеддингов и сохранение их в векторном хранилище ChromaDB.
    """
    # Создание эмбеддингов с помощью модели DeepPavlov/rubert-base-cased напрямую
    # Преобразование списка документов в список строк
    text_data = [doc.page_content for doc in texts]

    # Создание эмбеддингов для каждого текста
    embeddings = [create_embeddings(text) for text in text_data]
    
    # Создание векторного хранилища ChromaDB
    vectorstore = Chroma.from_embeddings(
        documents=texts,
        embedding=embeddings,
        persist_directory="chroma_db"
    )
    vectorstore.persist()
    return vectorstore


In [7]:
def call_api(prompt: str) -> str:
    url = "YOUR_API_ENDPOINT"  # Замените на фактический URL API
    headers = {
        "Content-Type": "application/json",
        # "Authorization": "Bearer YOUR_API_KEY"  # Если требуется аутентификация
    }
    payload = {
        "modelUri": "gpt://<идентификатор_каталога>/yandexgpt-lite",
        "completionOptions": {
            "stream": False,
            "temperature": 0.6,
            "maxTokens": "2000"
        },
        "messages": [
            {
                "role": "system",
                "text": "Найди ошибки в тексте и исправь их"
            },
            {
                "role": "user",
                "text": prompt
            }
        ]
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        result = response.json()
        # Извлечение сгенерированного текста из ответа
        generated_text = result.get('text', '')
        return generated_text
    else:
        raise Exception(f"API request failed with status code {response.status_code}: {response.text}")

In [8]:
def run_query(retriever, query):
    """
    Запуск поиска и генерация ответа с использованием API.
    """
    # Получение релевантных документов
    docs = retriever.get_relevant_documents(query)
    
    # Формирование контекста из документов
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # Создание полного запроса с учетом контекста
    prompt = f"Контекст:\n{context}\n\nВопрос:\n{query}"
    
    # Вызов API с предоставленным промптом
    response = call_api(prompt)
    
    # Возврат результата и исходных документов
    result = {
        "result": response,
        "source_documents": docs
    }
    return result

In [9]:
documents_folder = "..\documents"  # Путь к папке с документами
query = "Какие существуют типы источников выбросов? Привести примеры каждого источника."  # Пользовательский запрос

In [10]:
# Загрузка и разбиение документов
texts = load_and_split_documents(documents_folder)

In [11]:
texts, len(texts)

([Document(page_content='А.С. СТЕПАНОВСКИХ\n\n\n\n\nЭКОЛОГИЯ\n\n\n\n\n\nРекомендовано Министерством образования \nРоссийской Федерации в качестве учебника \nдля студентов высших учебных заведений\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nМосква» 2001\n\n\n\n\nУДК 574(075.8) \nББК 28.081 \nС 79\n\nРекомендовано Учебно-методическим центром «Профессиональный учебник» в качестве учебника для студентов высших учебных заведений\n\nРецензенты:\nд-р с.-х. наук, проф.. Засл. деятель науки РФ А.Г. Таскаева\n (Челябинский агроинженерный университет);\nд-р биол. наук, проф. T.В. Теплякова \n(Сибирский университет потребительской кооперации);\nд-р с.-х. наук, проф., Засл. деятель науки РФ В.А. Чулкина \n(Новосибирский аграрный университет)\n\nГлавный редактор издательства Н.Д. Эриашвили\n\nСтепановских А.С.\nС 79   Экология: Учебник для вузов. — М.: ЮНИТИ-ДАНА, 2001. - 703 с. \n\n\nISBN 5-238-00284-Х', metadata={'source': '..\\documents\\экология учебник.txt'}),
  Document(page_content='ISBN 5-238-00284-Х\n\n\n

In [14]:
# Создание векторного хранилища
vectorstore = create_vectorstore(texts)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum lengt

KeyboardInterrupt: 

In [ ]:
vectorstore

In [ ]:






# Настройка ретривера
retriever = vectorstore.as_retriever()

# Запуск запроса
result = run_query(retriever, query)

# Вывод ответа и исходных документов
print("Ответ:")
print(result["result"])
print("\nИсходные документы:")
for doc in result["source_documents"]:
print(doc.metadata["source"])

In [ ]:
# Основное выполнение
if __name__ == "__main__":
    # Параметры
    documents_folder = "./documents"  # Путь к папке с документами
    query = "В чем смысл жизни?"  # Пользовательский запрос
    
    # Загрузка и разбиение документов
    texts = load_and_split_documents(documents_folder)
    
    # Создание векторного хранилища
    vectorstore = create_vectorstore(texts)
    
    # Настройка ретривера
    retriever = vectorstore.as_retriever()
    
    # Запуск запроса
    result = run_query(retriever, query)
    
    # Вывод ответа и исходных документов
    print("Ответ:")
    print(result["result"])
    print("\nИсходные документы:")
    for doc in result["source_documents"]:
        print(doc.metadata["source"])


In [4]:
def create_vectorstore(texts, embedding_model_name):
    """
    Создание эмбеддингов и сохранение их в векторном хранилище ChromaDB.
    """
    # Создание эмбеддингов
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
    
    # Создание векторного хранилища ChromaDB
    vectorstore = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        persist_directory="chroma_db"
    )
    vectorstore.persist()
    return vectorstore